In [ ]:
from vj.voice.stt import WhisperSTT, MoonshineSTT, ParakeetSTT
from vj.voice.base import BaseSTT
from vj.evals.utils import (
    load_audio,
    ASSETS_DIR,
    RESULTS_DIR,
    chunk_audio,
    save_evals,
)

try:
    import jiwer
    import optuna
except ModuleNotFoundError:
    raise ModuleNotFoundError(
        "In order to run notebooks, please install dev dependencies with `uv sync --group dev`"
    )

import time
import io

2025-12-27 15:19:37.489157969 [W:onnxruntime:Default, device_discovery.cc:164 DiscoverDevicesForPlatform] GPU device discovery failed: device_discovery.cc:89 ReadFileContents Failed to open file: "/sys/class/drm/card0/device/vendor"
/home/marco/Training/Voxtral/.venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
def create_whisper_model(
    model: str,
    greedy: int,
    beam_size: int,
    patience: float,
    use_ctx: bool,
    n_threads: int,
) -> tuple[WhisperSTT, io.BytesIO, dict[str, int | str | float | bool], str]:

    params = {
        "no_context": use_ctx,
        "beam_search": {"beam_size": beam_size, "patience": patience},
        "greedy": {"best_of": greedy},
        "n_threads": n_threads,
        "print_progress": False,
    }

    return (
        *create_model(WhisperSTT),
        params,
        model,
    )


def create_model[ModelT: BaseSTT = BaseSTT](
    model_t: type[ModelT], **params
) -> tuple[ModelT, io.BytesIO]:

    buf = io.BytesIO()

    def save_response(text: str) -> None:
        buf.seek(0)
        buf.write(text.encode())

    return model_t(callback_fn=save_response, **params), buf

In [3]:
transformation = jiwer.Compose(
    [
        jiwer.ToLowerCase(),
        jiwer.RemovePunctuation(),
        jiwer.RemoveWhiteSpace(replace_by_space=True),
        jiwer.RemoveMultipleSpaces(),
        jiwer.Strip(),
    ]
)


def calculate_metrics(reference: str, hypothesis: str):
    """
    reference: The ground truth text
    hypothesis: What the model produced
    """
    # Clean both strings so capitalization/commas don't ruin the score
    ref_clean = transformation(reference)
    hyp_clean = transformation(hypothesis)

    wer = jiwer.wer(ref_clean, hyp_clean)
    wil = jiwer.wil(ref_clean, hyp_clean)  # Word Information Lost

    return {"wer": wer, "accuracy": 1 - wer, "wil": wil}

In [4]:
def run_evals(model: BaseSTT, trans_buf: io.BytesIO, **params):
    files = {
        "download.wav": "so what's going to happen now?",
        "download (1).wav": "you work amazingly but the transcription part is a little bit too slow",
    }
    scores = []
    print(f"----- Model: {model} -----")
    for file in files.keys():
        audio = load_audio(ASSETS_DIR / file)
        audio_duration = len(audio) / 16000  # Duration in seconds
        start_t = time.perf_counter()
        model.transcribe(audio, **params)
        end_t = time.perf_counter()
        latency = end_t - start_t
        rtf = audio_duration / latency
        trans = trans_buf.getvalue().decode()
        metrics = calculate_metrics(files[file], trans)
        scores.append(
            {
                "wer": metrics["wer"],
                "accuracy": metrics["accuracy"],
                "wil": metrics["wil"],
                "rtf": rtf,
                "latency": latency,
                "transcription": trans,
            }
        )
    return scores

In [5]:
stt, trans_buf, params, model = create_whisper_model("tiny", -1, -1, -1.0, True, 4)
scores = run_evals(stt, trans_buf)

whisper_init_from_file_with_params_no_state: loading model from '/home/marco/.local/share/pywhispercpp/models/ggml-small.bin'
whisper_init_with_params_no_state: use gpu    = 1
whisper_init_with_params_no_state: flash attn = 1
whisper_init_with_params_no_state: gpu_device = 0
whisper_init_with_params_no_state: dtw        = 0
whisper_init_with_params_no_state: devices    = 1
whisper_init_with_params_no_state: backends   = 1
whisper_model_load: loading model
whisper_model_load: n_vocab       = 51865
whisper_model_load: n_audio_ctx   = 1500
whisper_model_load: n_audio_state = 768
whisper_model_load: n_audio_head  = 12
whisper_model_load: n_audio_layer = 12
whisper_model_load: n_text_ctx    = 448
whisper_model_load: n_text_state  = 768
whisper_model_load: n_text_head   = 12
whisper_model_load: n_text_layer  = 12
whisper_model_load: n_mels        = 80
whisper_model_load: ftype         = 1
whisper_model_load: qntvr         = 0
whisper_model_load: type          = 3 (small)
whisper_model_load: 

----- Model: <voxtral.stt.whisper.WhisperSTT object at 0x753ad69ce510> -----


In [6]:
scores

[{'wer': 0.3333333333333333,
  'accuracy': 0.6666666666666667,
  'wil': 0.4666666666666667,
  'rtf': 1.1817821140801237,
  'latency': 2.589309792001586,
  'transcription': "So what's gonna happen now?"},
 {'wer': 0.0,
  'accuracy': 1.0,
  'wil': 0.0,
  'rtf': 2.9281507570129643,
  'latency': 2.7867417620000197,
  'transcription': 'You work amazingly but the transcription part is a little bit too slow.'}]

In [7]:
from typing import Literal, TypedDict


models = [
    "base",
    "base-q5_1",
    "base-q8_0",
    "small",
    "small-q5_1",
    "small-q8_0",
    "tiny",
    "tiny-q5_1",
    "tiny-q8_0",
]
greedies = Literal[0, 1, 2, -1]
beam_sizes = Literal[-1, 1, 2, 3, 4, 5]
patiences = Literal[-1.0, 0.0, 0.5, 1.0]
use_ctxs = Literal[False, True]
n_threads_l = Literal[4, 8, 16]

In [8]:
def whisper_objective(trial: optuna.Trial) -> tuple[float, float]:
    model = trial.suggest_categorical("model", models)
    greedy = trial.suggest_int("greedy", -1, 2)
    beam_size = trial.suggest_int("beam_size", -1, 5)
    patience = trial.suggest_float("patience", -1.0, 1.0, step=0.5)
    use_ctx = trial.suggest_int("use_ctx", 0, 1)
    n_threads = trial.suggest_int("n_threads", 4, 16, step=4)

    scores = []

    stt, trans_buf, params, _ = create_whisper_model(
        model, greedy, beam_size, patience, bool(use_ctx), n_threads
    )
    for score in run_evals(stt, trans_buf, **params):
        scores.append(score)

    return scores[1]["rtf"], scores[1]["wer"]

In [ ]:
study = optuna.create_study(
    storage=f"sqlite:///{RESULTS_DIR}/db.sqlite3",
    directions=["maximize", "minimize"],  # maximize rtf and minimize wer
)
study.optimize(whisper_objective, 100, n_jobs=4)

[I 2025-12-27 10:57:32,129] A new study created in RDB with name: no-name-97f34b9a-0a9f-4ea6-a410-a806bca8ce8f


whisper_init_from_file_with_params_no_state: loading model from '/home/marco/.local/share/pywhispercpp/models/ggml-small.bin'
whisper_init_with_params_no_state: use gpu    = 1
whisper_init_with_params_no_state: flash attn = 1
whisper_init_with_params_no_state: gpu_device = 0
whisper_init_with_params_no_state: dtw        = 0
whisper_init_with_params_no_state: devices    = 1
whisper_init_with_params_no_state: backends   = 1
whisper_model_load: loading model
whisper_model_load: n_vocab       = 51865
whisper_model_load: n_audio_ctx   = 1500
whisper_model_load: n_audio_state = 768
whisper_model_load: n_audio_head  = 12
whisper_model_load: n_audio_layer = 12
whisper_model_load: n_text_ctx    = 448
whisper_model_load: n_text_state  = 768
whisper_model_load: n_text_head   = 12
whisper_model_load: n_text_layer  = 12
whisper_model_load: n_mels        = 80
whisper_model_load: ftype         = 1
whisper_model_load: qntvr         = 0
whisper_model_load: type          = 3 (small)
whisper_model_load: 

----- Model: base-q5_1 -----
---- File: download.wav ----
--- Params: {'no_context': False, 'beam_search': {'beam_size': 1, 'patience': -0.5}, 'greedy': {'best_of': 1}, 'n_threads': 4, 'print_progress': False} ---
----- Model: small -----


whisper_init_from_file_with_params_no_state: loading model from '/home/marco/.local/share/pywhispercpp/models/ggml-small-q5_1.bin'
whisper_init_with_params_no_state: use gpu    = 1
whisper_init_with_params_no_state: flash attn = 1
whisper_init_with_params_no_state: gpu_device = 0
whisper_init_with_params_no_state: dtw        = 0
whisper_init_with_params_no_state: devices    = 1
whisper_init_with_params_no_state: backends   = 1
whisper_model_load: loading model
whisper_model_load: n_vocab       = 51865
whisper_model_load: n_audio_ctx   = 1500
whisper_model_load: n_audio_state = 768
whisper_model_load: n_audio_head  = 12
whisper_model_load: n_audio_layer = 12
whisper_model_load: n_text_ctx    = 448
whisper_model_load: n_text_state  = 768
whisper_model_load: n_text_head   = 12
whisper_model_load: n_text_layer  = 12
whisper_model_load: n_mels        = 80
whisper_model_load: ftype         = 9
whisper_model_load: qntvr         = 1
whisper_model_load: type          = 3 (small)
whisper_model_l

---- File: download.wav ----
--- Params: {'no_context': False, 'beam_search': {'beam_size': 4, 'patience': -1.0}, 'greedy': {'best_of': 2}, 'n_threads': 16, 'print_progress': False} ---
----- Model: small-q5_1 -----
---- File: download.wav ----
--- Params: {'no_context': True, 'beam_search': {'beam_size': 1, 'patience': 1.0}, 'greedy': {'best_of': -1}, 'n_threads': 8, 'print_progress': False} ---
----- Model: small -----
---- File: download.wav ----
--- Params: {'no_context': True, 'beam_search': {'beam_size': 4, 'patience': 1.0}, 'greedy': {'best_of': 1}, 'n_threads': 4, 'print_progress': False} ---


...

Evaluation finished in: 3.0643480139997337
Final trascription: So what's gonna happen now?ow?
Evaluated metrics: metrics={'wer': 1.0, 'accuracy': 0.0, 'wil': 1.0}
Evaluation finished in: 4.1429417170002125
Final trascription: So what's going to happen now?anscription part is a little bit slow.
Evaluated metrics: metrics={'wer': 1.1666666666666667, 'accuracy': -0.16666666666666674, 'wil': 0.6527777777777777}
---- File: download (1).wav ----
--- Params: {'no_context': True, 'beam_search': {'beam_size': 5, 'patience': -1.0}, 'greedy': {'best_of': -1}, 'n_threads': 4, 'print_progress': False} ---


[I 2025-12-27 11:01:46,764] Trial 95 finished with values: [0.8910649485294047, 1.0] and parameters: {'model': 'small-q8_0', 'greedy': -1, 'beam_size': 0, 'patience': 1.0, 'use_ctx': 0, 'n_threads': 8}.


Evaluation finished in: 7.271089979999942
Final trascription: So what's gonna happen now?
Evaluated metrics: metrics={'wer': 1.0, 'accuracy': 0.0, 'wil': 1.0}


[I 2025-12-27 11:01:46,967] Trial 99 finished with values: [0.31208607683821604, 0.15384615384615385] and parameters: {'model': 'base', 'greedy': -1, 'beam_size': 5, 'patience': -1.0, 'use_ctx': 1, 'n_threads': 4}.
[I 2025-12-27 11:01:46,970] Trial 96 finished with values: [0.87582146495099, 0.15384615384615385] and parameters: {'model': 'small-q5_1', 'greedy': 0, 'beam_size': 1, 'patience': 0.5, 'use_ctx': 1, 'n_threads': 16}.


Evaluation finished in: 7.146703154000079
Final trascription: You work amazingly, but the transcription part is a little bit slow.ow.
Evaluated metrics: metrics={'wer': 0.15384615384615385, 'accuracy': 0.8461538461538461, 'wil': 0.22435897435897445}
Evaluation finished in: 2.546622386999843
Final trascription: You work amazingly, but the transcription part is a little bit slow.ow.
Evaluated metrics: metrics={'wer': 0.15384615384615385, 'accuracy': 0.8461538461538461, 'wil': 0.22435897435897445}


In [9]:
def moonshine_obj(trial: optuna.Trial) -> tuple[float, float]:
    model = trial.suggest_categorical("model", ["tiny", "base"])

    scores = []
    stt, trans_buf = create_model(MoonshineSTT, **{"model": model})
    for score in run_evals(stt, trans_buf):
        scores.append(score)

    return scores[1]["rtf"], scores[1]["wer"]

In [14]:
moonshine_study = optuna.create_study(
    storage=f"sqlite:///{RESULTS_DIR}/moonshine_db.sqlite3",
    directions=["maximize", "minimize"],  # maximize rtf and minimize wer
)
moonshine_study.optimize(moonshine_obj, 10, n_jobs=4)

[I 2025-12-27 14:13:04,897] A new study created in RDB with name: no-name-452cfa95-2baf-4f11-ae32-4825e0b4db0a


----- Model: <voxtral.stt.moonshine.MoonshineSTT object at 0x749f85bb1d30> -----
----- Model: <voxtral.stt.moonshine.MoonshineSTT object at 0x749f841b1550> -----
----- Model: <voxtral.stt.moonshine.MoonshineSTT object at 0x749f85ba5c70> -----
----- Model: <voxtral.stt.moonshine.MoonshineSTT object at 0x749f85ba6b70> -----


[I 2025-12-27 14:13:09,779] Trial 3 finished with values: [20.106960008703545, 0.38461538461538464] and parameters: {'model': 'tiny'}.
[I 2025-12-27 14:13:09,943] Trial 0 finished with values: [15.635117649164055, 0.38461538461538464] and parameters: {'model': 'tiny'}.
[I 2025-12-27 14:13:10,090] Trial 2 finished with values: [13.620024659681384, 0.15384615384615385] and parameters: {'model': 'base'}.
[I 2025-12-27 14:13:10,841] Trial 1 finished with values: [6.124928412928033, 0.15384615384615385] and parameters: {'model': 'base'}.


----- Model: <voxtral.stt.moonshine.MoonshineSTT object at 0x749fb6f0a5b0> -----


[I 2025-12-27 14:13:12,218] Trial 4 finished with values: [6.490488522742609, 0.38461538461538464] and parameters: {'model': 'tiny'}.


----- Model: <voxtral.stt.moonshine.MoonshineSTT object at 0x74a02a10cad0> -----
----- Model: <voxtral.stt.moonshine.MoonshineSTT object at 0x74a02a10f1d0> -----
----- Model: <voxtral.stt.moonshine.MoonshineSTT object at 0x749f859ee780> -----
----- Model: <voxtral.stt.moonshine.MoonshineSTT object at 0x749f859ee200> -----


[I 2025-12-27 14:13:15,890] Trial 8 finished with values: [26.50542911744428, 0.38461538461538464] and parameters: {'model': 'tiny'}.
[I 2025-12-27 14:13:16,027] Trial 5 finished with values: [5.322828397504541, 0.15384615384615385] and parameters: {'model': 'base'}.
[I 2025-12-27 14:13:16,129] Trial 7 finished with values: [11.008488568650407, 0.15384615384615385] and parameters: {'model': 'base'}.
[I 2025-12-27 14:13:16,212] Trial 6 finished with values: [9.832843006445552, 0.15384615384615385] and parameters: {'model': 'base'}.


----- Model: <voxtral.stt.moonshine.MoonshineSTT object at 0x749f841f3750> -----


[I 2025-12-27 14:13:17,779] Trial 9 finished with values: [34.43327969741979, 0.15384615384615385] and parameters: {'model': 'base'}.


In [15]:
def parakeet_objective(trial: optuna.Trial) -> tuple[float, float]:
    model = trial.suggest_categorical(
        "model",
        [
            "nemo-parakeet-ctc-0.6b",
            "nemo-parakeet-rnnt-0.6b",
            "nemo-parakeet-tdt-0.6b-v2",
            "nemo-parakeet-tdt-0.6b-v3",
        ],
    )
    quant = trial.suggest_categorical("quant", [None, "int8"])
    stt, trans_buf = create_model(ParakeetSTT, **{"model": model, "quant": quant})

    scores = []
    for score in run_evals(stt, trans_buf):
        scores.append(score)
    return scores[1]["rtf"], scores[1]["wer"]

In [17]:
parakeet_study = optuna.create_study(
    storage=f"sqlite:///{RESULTS_DIR}/parakeet_db.sqlite3",
    directions=["maximize", "minimize"],  # maximize rtf and minimize wer
)
parakeet_study.optimize(parakeet_objective, 50, n_jobs=4)

[I 2025-12-27 16:25:46,525] A new study created in RDB with name: no-name-064f4994-e041-44d2-9f75-93ab9874d01d


----- Model: <voxtral.stt.parakeet.ParakeetSTT object at 0x753a201459d0> -----
----- Model: <voxtral.stt.parakeet.ParakeetSTT object at 0x753a2d3a6ed0> -----
----- Model: <voxtral.stt.parakeet.ParakeetSTT object at 0x753a20102ab0> -----
----- Model: <voxtral.stt.parakeet.ParakeetSTT object at 0x753a2d26ef90> -----


[I 2025-12-27 16:25:56,740] Trial 0 finished with values: [4.499938177137236, 0.0] and parameters: {'model': 'nemo-parakeet-rnnt-0.6b', 'quant': 'int8'}.
[I 2025-12-27 16:25:59,568] Trial 1 finished with values: [2.0827432865467963, 0.0] and parameters: {'model': 'nemo-parakeet-tdt-0.6b-v2', 'quant': 'int8'}.


----- Model: <voxtral.stt.parakeet.ParakeetSTT object at 0x753a2d232810> -----
----- Model: <voxtral.stt.parakeet.ParakeetSTT object at 0x753a200e9bb0> -----


[I 2025-12-27 16:26:02,114] Trial 2 finished with values: [1.4308803361108757, 0.0] and parameters: {'model': 'nemo-parakeet-rnnt-0.6b', 'quant': None}.
[I 2025-12-27 16:26:04,216] Trial 3 finished with values: [1.03451587991619, 0.0] and parameters: {'model': 'nemo-parakeet-rnnt-0.6b', 'quant': None}.


...

[I 2025-12-27 16:27:26,979] Trial 43 finished with values: [2.904153517835191, 0.0] and parameters: {'model': 'nemo-parakeet-rnnt-0.6b', 'quant': 'int8'}.
[I 2025-12-27 16:27:26,980] Trial 44 finished with values: [3.0742190589861673, 0.8461538461538461] and parameters: {'model': 'nemo-parakeet-tdt-0.6b-v3', 'quant': 'int8'}.


----- Model: <voxtral.stt.parakeet.ParakeetSTT object at 0x753a239056d0> -----


[I 2025-12-27 16:27:29,509] Trial 45 finished with values: [4.755376126665514, 0.07692307692307693] and parameters: {'model': 'nemo-parakeet-ctc-0.6b', 'quant': 'int8'}.


----- Model: <voxtral.stt.parakeet.ParakeetSTT object at 0x753a239a53d0> -----
----- Model: <voxtral.stt.parakeet.ParakeetSTT object at 0x753a2d406150> -----


[I 2025-12-27 16:27:31,801] Trial 46 finished with values: [3.954596375657883, 0.0] and parameters: {'model': 'nemo-parakeet-tdt-0.6b-v2', 'quant': None}.
[I 2025-12-27 16:27:32,863] Trial 47 finished with values: [6.382097266108188, 0.0] and parameters: {'model': 'nemo-parakeet-tdt-0.6b-v2', 'quant': 'int8'}.
[I 2025-12-27 16:27:32,944] Trial 48 finished with values: [5.831056883257508, 0.0] and parameters: {'model': 'nemo-parakeet-tdt-0.6b-v2', 'quant': 'int8'}.
[I 2025-12-27 16:27:32,999] Trial 49 finished with values: [9.59798681660592, 0.07692307692307693] and parameters: {'model': 'nemo-parakeet-ctc-0.6b', 'quant': 'int8'}.
